In [1]:
from pyspark import SparkContext, SparkConf, RDD
from pyspark.statcounter import StatCounter
from datetime import datetime

In [2]:
config = SparkConf()\
            .set("spark.driver.memory", "4g")\
            .set("spark.executor.memory", "8g")

In [3]:
sc = SparkContext(appName='taxi_duration_lowlevel', conf=config)

your 131072x1 screen size is bogus. expect trouble
25/03/31 17:47:12 WARN Utils: Your hostname, DESKTOP-0H87CFM resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/31 17:47:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/31 17:47:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
#Convert data into rdd
raw_rdd = sc.textFile('train.csv')

#Remove the csv header row
header = raw_rdd.first()

#Strip the header row
rdd_no_header = raw_rdd.filter(lambda row: row != header)

In [5]:
def vectorize(row: str):
    """Vectorize the features and labels."""

    #Split the string for elements, excluding the first feature (`id`)
    values = [item for item in row.split(',')]

    #Split the timestamps features into time elements
    pickup_time = datetime.strptime(values[2], '%Y-%m-%d %H:%M:%S').strftime('%Y:%m:%d:%H:%M:%S').split(':')
    dropoff_time = datetime.strptime(values[3], '%Y-%m-%d %H:%M:%S').strftime('%Y:%m:%d:%H:%M:%S').split(':')
    
    #Encode the `store_and_fwd_flag` into binary values
    values[9] = 1 if values[9] == 'Y' else 0

    #Cast strings into number
    raw_vector = [float(value) for value in [values[1]] + pickup_time + dropoff_time + values[4:]]
    
    #Build and return the LabeledPoint-like row
    return {'label': raw_vector[-1], 'features' : raw_vector[:-1]}

rdd = rdd_no_header.map(vectorize).repartition(8).cache()

In [6]:
class DecisionTreeRegressor:
    def __init__(self, max_depth = 1):
        self.max_depth = max_depth
        self.rules = None
        pass



    def set_maxDepth(self, depth):
        self.max_depth = depth



    def fit(self, train_rdd: RDD):
        self.num_features = len(rdd.first()['features'])
        self.usable_features = [i for i in range(self.num_features)]
        self.rules = self.__build_rule_tree_recursive(train_rdd, self.usable_features)

    

    def transform(self, rdd: RDD):
        
        pass

    def display_rule_tree(self):
        def display_tree_recusive(rules: dict, indent = 0):
            if not rules:
                return 
            print(f"{indent * ' '}if features[{rules['split_feature']}] <= {rules['split_point']}")
            display_tree_recusive(rules['left'], indent + 5)
            display_tree_recusive(rules['right'], indent + 5)

        display_tree_recusive(self.rules)

    
    def __find_best_split(self, rdd: RDD, usable_features: list): 

            
        def find_feature_best_split(values: list[list[float ] ], parent_info: StatCounter):
            sorted_values = sorted(values)
            
            best_split, best_var_reduction = None, -float("inf")

            if any(sorted_values[i][0] != sorted_values[0][0] for i in range(1, parent_info.count())):
                parent_pow_sum = parent_info.variance() + parent_info.mean() ** 2

                left_sum, left_pow_sum, left_count = 0, 0, 0
                
                
                previous_feature_val =  None
                for feature_val, label in sorted_values:
                    left_sum += label
                    left_pow_sum += label ** 2
                    left_count += 1

                    if not previous_feature_val:
                        previous_feature_val = feature_val
                        continue

                    right_count = parent_info.count() - left_count

                    if not right_count:
                        continue

                    right_sum = parent_info.sum() - left_sum
                    right_pow_sum = parent_pow_sum - left_pow_sum

                    
                    left_var = left_pow_sum / left_count - (left_sum / left_count)**2 if left_count > 0 else 0
                    right_var = right_pow_sum / right_count - (right_sum / right_count)**2 if right_count > 0 else 0

                    var_reduction = parent_info.variance() - left_var * (left_count / parent_info.count()) - right_var * (right_count / parent_info.count())

                    if var_reduction > best_var_reduction:
                        best_split = (previous_feature_val + feature_val) / 2
                        best_var_reduction = var_reduction


                    previous_feature_val = feature_val
                    

            return best_var_reduction, best_split

        def partition_feature_grouping(partition, usable_features: list):
            feature_stats = {}

            for row in partition:
                for feature in usable_features:
                    # Store feature-wise values in a dictionary
                    if feature not in feature_stats:
                        feature_stats[feature] = []

                    feature_stats[feature].append((row['features'][feature], row['label']))

            results = []
            
            for feature, values in feature_stats.items():
                results.append((feature, values))
        
            return iter(results)


        parent_info = rdd.mapPartitions(lambda partition: [StatCounter([row['label'] for row in partition])], True)\
                        .reduce(lambda stat1, stat2: stat1.mergeStats(stat2))
        
        max_info = rdd.mapPartitions(lambda partition: partition_feature_grouping(partition, usable_features), True)\
                      .reduceByKey(lambda l1, l2: l1 + l2)\
                      .mapValues(lambda values: find_feature_best_split(values, parent_info))\
                      .max(key= lambda tupl: tupl[1][0])

        return max_info[0], max_info[1][1]




    def __build_rule_tree_recursive(self, parent: RDD, usable_features: list, depth = 0):
        if depth == self.max_depth or parent.isEmpty():
            return None
    
        split_feature, split_point = self.__find_best_split(parent, usable_features)




        if split_feature == None or split_point == None:
            return None


        def splitter(iterator, split_feature: int, split_point: float, operand):
            for row in iterator:
                if operand(row['features'][split_feature], split_point):
                    yield row




        left_rdd = parent.mapPartitions(lambda iterator: splitter(iterator, split_feature, split_point, lambda a,b: a <= b))
        right_rdd = parent.mapPartitions(lambda iterator: splitter(iterator, split_feature, split_point, lambda a,b: a > b))
        
        new_features = [v for v in usable_features if v != split_feature]

        return {
            'split_feature' : split_feature,
            'split_point' : split_point,
            'left' : self.__build_rule_tree_recursive(left_rdd, new_features, depth + 1),
            'right' : self.__build_rule_tree_recursive(right_rdd, new_features.copy(), depth + 1)
        }
    




In [7]:
estimator = DecisionTreeRegressor(10)
estimator.fit(rdd)

estimator.display_rule_tree()

25/03/31 17:47:16 WARN BlockManager: Task 7 already completed, not releasing lock for rdd_7_0
25/03/31 17:47:17 WARN BlockManager: Task 8 already completed, not releasing lock for rdd_7_0
25/03/31 17:47:34 WARN BlockManager: Task 33 already completed, not releasing lock for rdd_7_0
25/03/31 17:47:47 WARN BlockManager: Task 58 already completed, not releasing lock for rdd_7_0
25/03/31 17:48:03 WARN BlockManager: Task 83 already completed, not releasing lock for rdd_7_0
25/03/31 17:48:15 WARN BlockManager: Task 108 already completed, not releasing lock for rdd_7_0
25/03/31 17:48:23 WARN BlockManager: Task 133 already completed, not releasing lock for rdd_7_0
25/03/31 17:48:34 WARN BlockManager: Task 158 already completed, not releasing lock for rdd_7_0
25/03/31 17:48:42 WARN BlockManager: Task 183 already completed, not releasing lock for rdd_7_0
25/03/31 17:48:53 WARN BlockManager: Task 208 already completed, not releasing lock for rdd_7_0
25/03/31 17:49:04 WARN BlockManager: Task 233 a

if features[0] <= 2.0
     if features[9] <= 31.0
          if features[2] <= 6.0
               if features[8] <= 6.0
                    if features[13] <= 6.0
                         if features[4] <= 23.0
                              if features[10] <= 23.0
                                   if features[3] <= 31.0
                                        if features[6] <= 59.0
                                             if features[5] <= 59.0
                         if features[3] <= 15.5
                              if features[14] <= -74.10657501220703
                    if features[4] <= 22.0
                         if features[14] <= -73.93830490112305
                              if features[11] <= 9.0
                                   if features[12] <= 52.0
                                        if features[13] <= 1.0
                                             if features[10] <= 14.5
                         if features[6] <= 59.0
                              if 